In [1]:
import tensorflow as tf
import os
import cv2
import pickle
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense,Flatten,AveragePooling2D,Dropout

In [2]:
Bmodel = ResNet50(weights = 'imagenet', include_top = False, input_shape = (150, 150, 3))

In [14]:
hmodel = Bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(512, activation="relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(1,activation = "sigmoid")(hmodel)

In [15]:
model = tf.keras.Model( Bmodel.input, hmodel)

for layer in Bmodel.layers:
  layer.trainable = False

In [20]:
from tensorflow.keras.optimizers import SGD
epoch = 25
optimizer = SGD(lr = 0.01)
model.compile(optimizer = optimizer, 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
base_dir = 'data_faces2'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_drowsiness_dir = os.path.join(train_dir, 'drowsiness') # Directory with our training drowsiness pictures
train_undrowsiness_dir = os.path.join(train_dir, 'undrowsiness') # Directory with our training undrowsiness pictures
validation_drowsiness_dir = os.path.join(validation_dir, 'drowsiness') # Directory with our validation drowsiness pictures
validation_undrowsiness_dir = os.path.join(validation_dir, 'undrowsiness')# Directory with our validation undrowsiness pictures

train_drowsiness_fnames = os.listdir(train_drowsiness_dir)
train_undrowsiness_fnames = os.listdir(train_undrowsiness_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   brightness_range=None,
                                   shear_range=0.2,
                                   zoom_range=0.3,
                                   channel_shift_range=0.0,
                                   fill_mode="nearest",
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    shuffle=True,
                                                    batch_size = 30,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 10,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(train_generator.classes)

Found 1497 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
{'drowsiness': 0, 'undrowsiness': 1}
{'drowsiness': 0, 'undrowsiness': 1}
[0 0 0 ... 1 1 1]


In [22]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = epoch,
            verbose = 1)

Epoch 1/25
50/50 [==============================] - 484s 10s/step - loss: 0.6936 - accuracy: 0.5170 - val_loss: 0.6960 - val_accuracy: 0.4261
Epoch 2/25
50/50 [==============================] - 474s 9s/step - loss: 0.6926 - accuracy: 0.5130 - val_loss: 0.6994 - val_accuracy: 0.4261
Epoch 3/25
50/50 [==============================] - 471s 9s/step - loss: 0.6944 - accuracy: 0.5077 - val_loss: 0.6995 - val_accuracy: 0.4261
Epoch 4/25
50/50 [==============================] - 538s 11s/step - loss: 0.6954 - accuracy: 0.4997 - val_loss: 0.6933 - val_accuracy: 0.4261
Epoch 5/25
50/50 [==============================] - 518s 10s/step - loss: 0.6931 - accuracy: 0.5210 - val_loss: 0.7014 - val_accuracy: 0.4261
Epoch 6/25
50/50 [==============================] - 516s 10s/step - loss: 0.6936 - accuracy: 0.5137 - val_loss: 0.7013 - val_accuracy: 0.4261
Epoch 7/25
50/50 [==============================] - 512s 10s/step - loss: 0.6933 - accuracy: 0.5084 - val_loss: 0.7038 - val_accuracy: 0.4261
Epoch 8/

KeyboardInterrupt: 